In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchain-ollama langchain-experimental neo4j tiktoken yfiles_jupyter_graphs python-dotenv json-repair langchain-openai langchain_core langchain-neo4j

Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain_core.runnables import  RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
from langchain_neo4j import Neo4jGraph
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_ollama import OllamaEmbeddings, ChatOllama
import os
from neo4j import Driver

from dotenv import load_dotenv

load_dotenv(override=True)

True

In [2]:
graph = Neo4jGraph()

In [3]:
loader = TextLoader(file_path="dummytext.txt")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=100)
documents = text_splitter.split_documents(documents=docs)

In [4]:
llm = ChatOllama(model="llama3.1", temperature=0)
# llm = OllamaFunctions(model="llama3.2", temperature=0)
llm_transformer = LLMGraphTransformer(llm=llm)



In [5]:
graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [6]:
graph_documents

[GraphDocument(nodes=[Node(id="Amico'S Family", type='Family', properties={}), Node(id='Legacy', type='Concept', properties={}), Node(id='Love', type='Emotion', properties={}), Node(id='Tradition', type='Concept', properties={})], relationships=[], source=Document(metadata={'source': 'dummytext.txt'}, page_content='1. The Story of Amicoâ€™s Family: A Legacy of Love and Tradition')),
 GraphDocument(nodes=[Node(id='Santa Caterina', type='Location', properties={}), Node(id='Caruso Family', type='Person', properties={})], relationships=[], source=Document(metadata={'source': 'dummytext.txt'}, page_content="In the idyllic village of Santa Caterina, amidst the rolling hills and sun-kissed landscapes of Sicily, lies the genesis of the Caruso family, a lineage intertwined with the island's rich culinary tapestry. The Carusos were not mere inhabitants of the land; they were the keepers of a culinary heritage that spanned generations. Each family member contributed their unique flair, crafting a

In [10]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [8]:
def showGraph():
    driver= GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget=GraphWidget(graph=session.run("MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t").graph())
    widget.node_label_mapping = 'id'
    return widget  

showGraph()

GraphWidget(layout=Layout(height='500px', width='100%'))

In [42]:
embeddings = OllamaEmbeddings(
    model="nomic-embed-text",
)

vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)
vector_retriever = vector_index.as_retriever()

ValueError: Index with name vector already exists.The provided embedding function and vector index dimensions do not match.
Embedding function dimension: 768
Vector index dimension: 1024

In [10]:
driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))

def create_fulltext_index(tx):
    query = '''
    CREATE FULLTEXT INDEX `fulltext_entity_id` 
    FOR (n:__Entity__) 
    ON EACH [n.id];
    '''
    tx.run(query)
    
# Function to execute the query
def create_index():
    with driver.session() as session:
        session.execute_write(create_fulltext_index)
        print("Fulltext index created successfully.")

# Call the function to create the index
try:
    create_index()
except Exception as e:
    print(e)
    pass

# Close the driver connection
driver.close()

{code: Neo.ClientError.Schema.EquivalentSchemaRuleAlreadyExists} {message: An equivalent index already exists, 'Index( id=7, name='fulltext_entity_id', type='FULLTEXT', schema=(:__Entity__ {id}), indexProvider='fulltext-1.0' )'.}


In [21]:
class Entities(BaseModel):
    """Identifying information about entities."""

    names: list[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)


entity_chain = llm.with_structured_output(Entities)

In [22]:
entity_chain.invoke("Who is Antonio Caruso?")

Entities(names=['Antonio Caruso'])

In [23]:
def generate_full_text_query(input: str) -> str:
    words = [el for el in remove_lucene_chars(input).split() if el]
    if not words:
        return ""
    full_text_query = " AND ".join([f"{word}~2" for word in words])
    print(f"Generated Query: {full_text_query}")
    return full_text_query.strip()


# Fulltext index query
def graph_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""
    entities = entity_chain.invoke(question)
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('fulltext_entity_id', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": entity},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [24]:
print(graph_retriever("Who is Nonna Lucia?"))

In [25]:
def full_retriever(question: str):
    graph_data = graph_retriever(question)
    vector_data = [el.page_content for el in vector_retriever.invoke(question)]
    final_data = f"""Graph data:
        {graph_data}
        vector data:
        {"#Document ". join(vector_data)}
        """
    return final_data

In [26]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
        {
            "context": full_retriever,
            "question": RunnablePassthrough(),
        }
    | prompt
    | llm
    | StrOutputParser()
)

In [27]:
chain.invoke(input="Who is Nonna Lucia? Did she teach anyone about restaurants or cooking?")

'{ "Nonna Lucia" : "Lucia, Antonio\'s sister and Amico\'s grandmother. She was a culinary sage who taught her grandchildren the art of Sicilian cooking." }'

In [28]:
chain.invoke(input="Who is Antonio Caruso? What is the relationship between Antonio Caruso and Nonna Lucia?")

'{ "Antonio Caruso" : "A chef who hosts workshops in Rome, sharing his innovative techniques. He is also Nonna Lucia\'s son." }'

In [29]:
chain.invoke(input="Who is Amico Caruso? List all his family members.")

'{ "Amico Caruso" : \n  -1.2 \n  }\n \n  \n\n\n\n\n\n  \n\n\n\n\n\n  \n\n\n\n\n\n  \n\n\n\n\n\n  \n\n\n\n\n\n  \n\n\n\n\n\n  \n\n\n\n\n\n  \n\n\n\n\n\n  \n\n\n\n\n\n  \n\n\n\n\n\n'

In [31]:
chain.invoke(input="Print the entire context given")

'{ "Conclusion": "In the idyllic village of Santa Caterina, amidst the rolling hills and sun-kissed landscapes of Sicily, lies the genesis of the Caruso family, a lineage intertwined with the island\'s rich culinary tapestry. The Carusos were not mere inhabitants of the land; they were the keepers of a culinary tradition that brought people from different backgrounds together.", "In the idyllic village of Santa Caterina, amidst the rolling hills and sun-kissed landscapes of Sicily, lies the genesis of the Caruso family, a lineage intertwined with the island\'s rich culinary tapestry. The Carusos were not mere inhabitants of #Document"  : "bringing people from different backgrounds together." }'